In [ ]:
#default_exp typing

# Typing
> Custom types used throughout the library

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#export
import enum
import typing
from typing import Any
from functools import wraps, partial

In [ ]:
#export
class _MemberGenericAlias(typing._GenericAlias, _root=True):
    def copy_with(self, params):
        return Member[params]
    
    def __eq__(self, other):
        if not isinstance(other, _MemberGenericAlias):
            return NotImplemented
        return set(self.__args__) == set(other.__args__)
    
    def __hash__(self):
        return hash(frozenset(self.__args__))
    
    def __repr__(self): 
        args = list(self.__args__)
        if isinstance(args[0], str):
            args[0] = f'"{args[0]}"'
        if len(args) == 2:
            # Value, Documentation
            return f'Member[value={args[0]}, doc="{args[1]}"]'
        else:
            return f'Member[doc={args[0]}]'
    
    def __instancecheck__(self, obj):
        return self.__subclasscheck__(type(obj))
    
    def __subclasscheck__(self, cls):
        for arg in self.__args__:
            if issubclass(cls, arg):
                return True
            
    def __reduce__(self):
        func, (origin, args) = super().__reduce__()
        return func, (Member, args)
    
    @property
    def value(self):
        args = self.__args__
        if len(self.__args__) == 2:
            return args[0]
        return None
    
    @property
    def doc(self):
        args = self.__args__
        if len(self.__args__) > 0:
            return args[-1]
        return None

In [ ]:
#export
@typing._SpecialForm
def Member(self, parameters):
    """Member type; Member[X, Y] means value of X, documentation of Y
    
    Used to quickly write documented Enums that may 
    have a value of the member name, in lower case.
    
    To define a member type:
      - Member["Some documentation"]
      - Member["some_value", "Some documentation"]
  
    If a Member has a single value, the Enum value will be the member name, in lower case,
    and the documentation will be that value
    
    If two, then the Enum value will be the first, with its documentation the second
    
    These attributes can be accessed with Member.value or Member.doc
    """
    if parameters == ():
        raise TypeError("Cannot take a Member without documentation")
    if not isinstance(parameters, tuple):
        parameters = (parameters,)
    return _MemberGenericAlias(self, parameters)

In [ ]:
t = Member["Some documentation"]
test_eq(t.doc, "Some documentation")
test_eq(t.value, None)
test_eq(t.__repr__(), 'Member[doc="Some documentation"]')

t2 = Member["My value", "Some documentation"]
test_eq(t2.doc, "Some documentation")
test_eq(t2.value, "My value")
test_eq(t2.__repr__(), 'Member[value="My value", doc="Some documentation"]')

t3 = Member[3, "Some documentation"]
test_eq(t3.doc, "Some documentation")
test_eq(t3.value, 3)
test_eq(t3.__repr__(), 'Member[value=3, doc="Some documentation"]')

In [ ]:
#export
class _DocumentedEnum(enum.Enum):
    """
    An `Enum` capabile of having its members have docstrings, compatible with the functional API
    """
    def __init__(self, *args):
        """
        Creates a generic enumeration with potential assigning of a member docstring

        Should be passed in the form of:
          value, docstring
        """
        if len(args) == 2:
            # Have value, docstring
            self._value_, self.__doc__ = args
        else:
            # Just value
            self._value_ = args[0]
    
    def __str__(self): return str(self.value)
    def __eq__(self, other): return getattr(other, "value", other) == self.value
    def __ne__(self, other): return getattr(other, "value", other) != self.value

In [ ]:
#hide
_da = [["addition", ("addition", "Sum of two numbers")], ["subtraction", ("subtraction")]]

_d = _DocumentedEnum("test_enum", _da)
test_eq(hasattr(_d, "addition"), True)
test_eq(str(_d.addition), "addition")
test_eq(_d.addition.__doc__, "Sum of two numbers")
test_eq(_d.addition == "addition", True)

test_eq(str(_d.subtraction), "subtraction")
test_eq(_d.subtraction.__doc__, "An enumeration.")
test_eq(_d.subtraction != "addition", True)

In [ ]:
#export
def _assign_annotations(cls):
    """
    Creates a `_DocumentedEnum` based on annotations in `cls`
    """
    cls_annotations = cls.__dict__.get("__annotations__", {})
    names = []
    for name, typ in list(cls_annotations.items()):
        member = [name]
        value = getattr(typ, "value", None)
        if value is None:
            value = name.lower()
            if typ == Member:
                names.append([name, value])
                continue
        if getattr(typ, "doc", None) is not None:
            member.append((value, typ.doc))
        names.append(member)
    new_cls = _DocumentedEnum(value=cls.__name__, names=names)
    new_cls.__doc__ = cls.__doc__
    return new_cls

In [ ]:
#export
def enumify(cls=None, *, special:list=None):
    """
    A decorator to turn `cls` into an Enum class with member values as property names, and potentially with documentation
    
    Should be documented with the `Member` type with the following annotation:
    ```python
    from fastreinference.typing import Member
    @enumify
    class MyClass:
      NAME:Member["Some documented enum value"]
      name_two:Member # An undocumented enum value
      name_three:Memeber["A value", "Some documentation"]
    ```
    """
    def wrap(cls): return _assign_annotations(cls)
    if cls is None:
        return partial(enumify)
    return wrap(cls)

In [ ]:
@enumify
class DaysOfWeek:
    MONDAY:Member["First day of the week"]
    TUESDAY:Member
    WEDNESDAY:Member["Wed", "Third day of the week"]
    THURSDAY:Member[0, "Fourth day of the week"]
    
test_eq(DaysOfWeek.MONDAY, "monday")
test_eq(DaysOfWeek.MONDAY.__doc__, "First day of the week")
test_eq(DaysOfWeek.TUESDAY, "tuesday")
test_eq(DaysOfWeek.TUESDAY.__doc__, None)
test_eq(DaysOfWeek.WEDNESDAY, "Wed")
test_eq(DaysOfWeek.WEDNESDAY.__doc__, "Third day of the week")
test_eq(DaysOfWeek.THURSDAY, 0)
test_eq(DaysOfWeek.THURSDAY.__doc__, "Fourth day of the week")